In [1]:
from selenium import webdriver
import time
import pandas as pd
import json
import src.scrapping as sr

In [188]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()

In [189]:
driver.get('http://jv.acb.com/es/101266/cartadetiro')
driver.implicitly_wait(10)

### Getting the data from "Jornada Virtual ACB"

In [225]:
final = sr.season(1, 35)
data_items = final.items()
data_list = list(data_items)

df = pd.DataFrame(data_list)

In [ ]:
df.head

In [227]:
df.to_csv("./data/shots_v1.csv")

### Getting general data about every player.

In [ ]:
driver.get('https://basketball.realgm.com/international/league/4/Spanish-ACB/team/649/Acunsa-GBC/rosters/2021')
driver.find_element_by_class_name('Frame-sc-1d4hofp-0.czVBsw').click()

In [ ]:
columns_data = sr.players_name("tablesaw.tablesaw-swipe.tablesaw-sortable")

In [ ]:
players_data = sr.table_stats("tablesaw.tablesaw-swipe.tablesaw-sortable")

In [ ]:
general_final = np.array_split(players_data,364)
general_stats = pd.DataFrame(general_final, columns = columns_data)
general_stats.drop(columns = ["#", "Birth City", "NBA Draft Status", "Nationality"], inplace = True)

### Getting all the stats per minute of every player in this season (2020-21).

In [ ]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
driver.implicitly_wait(10)

In [ ]:
driver.get('https://basketball.realgm.com/international/league/4/Spanish-ACB/team/649/Acunsa-GBC/stats/2021/Per_Minute/All/All/points/All/desc/1/Regular_Season')
driver.find_element_by_class_name('Frame-sc-1d4hofp-0.czVBsw').click() #We use it to accept the cookies

In [ ]:
#Getting the headers from stats in-game of every player. 

headers_player_stats = sr.players_name("tablesaw.compact.tablesaw-swipe")

In [ ]:
per_minute_stats = sr.table_stats("tablesaw.compact.tablesaw-swipe")

In [ ]:
#The result of "no_players" will be the number of players whose we have info about. 

no_players = len(per_minute_stats) / 23
final = np.array_split(per_minute_stats, no_players)
acb_dataset_per_minute = pd.DataFrame(final, columns = headers_player_stats)

### Getting all the stats averaged of every player in this season (2020-21).

In [ ]:
driver.get('https://basketball.realgm.com/international/league/4/Spanish-ACB/team/649/Acunsa-GBC/stats/2021/Averages/All/All/points/All/desc/1/Regular_Season')
driver.find_element_by_class_name('Frame-sc-1d4hofp-0.czVBsw').click()

In [ ]:
average_stats = sr.table_stats("tablesaw.compact.tablesaw-swipe")

In [ ]:
#The result of "no_players" will be the number of players whose we have info about. 

no_players = len(average_stats) / 23
average_final = np.array_split(average_stats, no_players)
average_stats = pd.DataFrame(average_final, columns = headers_player_stats)

### Merging general stats with stats tables

In [ ]:
final_per_minute = acb_dataset_per_minute.merge(general_stats, how  = "inner", left_on = "Player", right_on = "Player")

In [ ]:
final_per_minute.to_csv("./data/stats_per_minute.csv")

In [ ]:
final_average = average_stats.merge(general_stats, how  = "inner", left_on = "Player", right_on = "Player")

In [ ]:
final_average.to_csv("./data/acb_stats.csv")